## Workflow

Essense of ML and Deep Learning is to take some data and write an algorithm to learn from it and make predictions on new data

### Steps in this process
1. Collect data
2. Build a model
3. Train the model
4. Make predictions
5. Evaluate and improve
6. Save and load the model

In [ ]:
import torch 
from torch import nn
from matplotlib import pyplot as plt

Let us prepare some data, let us take a simple linear function
y = 6x + 2

We will use this function to generate some data and then try to fit a model to this data, so basically for data we are using this and then writing an algorithm to fit a model to this data

In real world, we will have some data and we will try to fit a model to this data, but for now we are generating the data and then fitting a model to this data


In [ ]:
# Generating data

# Let us generate some data
x = torch.arange(0,1,0.03).float().unsqueeze(1)
y = 2 * x + 8

# Let us visualize the data
plt.scatter(x, y,color='blue')
plt.show()

In [ ]:
# Split the data into training and testing data
n = len(x)

# 80% of the data will be used for training
n_train = int(0.8 * n)
x_train = x[:n_train]
y_train = y[:n_train]

# 20% of the data will be used for testing
x_test = x[n_train:]
y_test = y[n_train:]

In [ ]:
# Define plot function
def plot_graph(x_train,y_train,x_test,y_test,predictions=None):
    plt.scatter(x_train, y_train,
                color='blue',s=4, label='Training Data')
    plt.scatter(x_test, y_test,
                color='green',s=4, label='Testing Data')
    
    if predictions is not None:
        plt.scatter(x_test, predictions,s=4, color='red', label='Predictions')
    plt.legend()
    plt.show()

In [ ]:
plot_graph(x_train, y_train, x_test, y_test)

Define Linear Regression Model

Why?
We want to find the best line that fits the data. We can represent the line using a linear equation y = wx + b. We want to find the best values of w and b that fits the data

How do we know that we have to use a linear regression model?
We know that we have to use a linear regression model because we want to find the best line that fits the data. We can represent the line using a linear equation y = wx + b. We want to find the best values of w and b that fits the data

Why do we use nn.Module?
We use nn.Module because we want to create a custom model. We want to create a model which is basically sub class of nn.Module. We want to create a model which is not already present in pytorch


In [ ]:
class LinearRegression(nn.Module):
    def __init__(self):
        # why super().__init__()?
        # super() function returns the parent class, in this case, the parent class is nn.Module, so we are calling the constructor of the parent class
        super().__init__()

        # Why nn.Linear(1, 1)?
        # We are creating a linear model with 1 input and 1 output
        self.linear = nn.Linear(1, 1)

        # Or we can also write
        # self.weight = nn.Parameter(torch.randn(1, 1))
        # self.bias = nn.Parameter(torch.randn(1))


    def forward(self, x):
        # what is the output of this function?
        # The output of this function is the output of the linear layer which is the prediction of the model for the given input
        return self.linear(x)
        # this linear will be like y = wx + b so here x is an tensor so output will be tensor

    
    # why forward function?
    # forward function is called when we call the model object with the input data, it is used to define the forward pass of the model, and where ever we use nn.Module, we need to define the forward function

In [ ]:
# Set the manual seed: Why? For reproducibility of the results
torch.manual_seed(42)

# Create the model
model = LinearRegression()

# See the parameters
print(list(model.parameters()))



<strong>What does require_grad mean?</strong><br>

`require_grad` is a flag that allows us to compute the gradients of the parameters.If we set it to True, we can compute the gradients, if we set it to False, we cannot compute the gradients


<strong>Why do we need to keep track of the gradients?</strong><br>

We need to keep track of the gradients to update the parameters of the model using the gradients of the loss function

Steps to train the model
1. Define the loss function
2. Define the optimizer
3. Train the model

In [ ]:
# Define the loss function
loss = nn.L1Loss() # L1 loss is also known as Mean Absolute Error

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # SGD is Stochastic Gradient Descent


What is Stochastic Gradient Descent?<br>
Stochastic Gradient Descent is an optimization algorithm used to minimize some function by iteratively moving in the direction of steepest descent as defined by the negative of the gradient

lr is the learning rate, it is a hyperparameter, it is the rate at which the model learns i.e how much the modal tweaks the weights and biases to minimize the loss
If the learning rate is too high, the model will not converge, if the learning rate is too low, the model will take a long time to converge

What is an hyperparameter?<br>
Hyperparameters are the parameters that are not learned by the model, but are set by the user, for example, learning rate, number of epochs, batch size, etc

What is an epoch?<br>
An epoch is when an entire dataset is passed forward and backward through the neural network exactly once, its like one complete cycle of the dataset

What is batch size?<br>
Batch size is the number of samples that will be passed through the neural network at once, for example, if the batch size is 32, then 32 samples will be passed through the neural network at once

What is a loss function?<br>
A loss function is a function that measures how well the model is performing, it measures the difference between the predicted value and the actual value

More info on loss functions: <br>
* https://neptune.ai/blog/pytorch-loss-functions#Mean-Absolute-Error 
* https://shiva-verma.medium.com/understanding-different-loss-functions-for-neural-networks-dd1ed0274718

In [ ]:
# Train the model
n_epochs = 100

# Store the loss values
epoch_data = []
train_loss_values = []
test_loss_values = []

for epoch in range(n_epochs):
    # Forward pass
    y_pred = model(x_train)
    
    # Compute the loss
    l = loss(y_pred, y_train)
    
    # Zero the gradients
    optimizer.zero_grad()
    
    # Backward pass
    l.backward()
    
    # Update the parameters
    optimizer.step()
    
    # Test the model
    model.eval()
    with torch.inference_mode():
        y_test_pred = model(x_test)
        test_loss = loss(y_test_pred, y_test)

    # Print the loss for every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Training Loss: {l.item()}, Test Loss: {test_loss.item()}')
        
        # Save the loss for plotting
        epoch_data.append(epoch)
        train_loss_values.append(l.item())
        test_loss_values.append(test_loss.item())


# Plot the loss
plt.plot(epoch_data, train_loss_values, label='train')
plt.plot(epoch_data, test_loss_values, label='test')
plt.legend()
plt.show()

What does forward pass do?<br>
Forward pass computes the prediction of the model for the given input data by passing the input data through the model

What is use of zero_grad()? <br>
zero_grad() is used to zero the gradients of the parameters of the model, so that the gradients do not accumulate over the iterations of the training loop

x_train is a tensor right, will all the input data be passed at once? <br>
Yes, x_train is a tensor, and all the input data will be passed at once, because the model is designed to take the entire input data at once, and the model is capable of handling the entire input data at once

But in forward pass, we are not passing the input data to the model, then how is the input data passed to the model?  <br>
In PyTorch, the input data is passed to the model when we call the model object with the input data, for example, when we call model(x_train), the input data x_train is passed to the model, and the forward function of the model is called with the input data

What is gradient descent? <br>
Gradient Descent is an optimization algorithm used to minimize the loss function of the model by updating the parameters of the model in the opposite direction of the gradient of the loss function with respect to the parameters
Gradient is the derivative of the loss function with respect to the parameters of the model, and in multivariable calculus, gradient is a vector that points in the direction of the steepest increase of the function. This is nothing but the derivative of the function with respect to each parameter. like 
For example z= f(x,y) then gradient of z with respect to x is dz/dx and gradient of z with respect to y is dz/dy

So basically after loss is calculated, we are calculating the gradient of the loss function with respect to the parameters of the model, and then we are updating the parameters of the model in the opposite direction of the gradient to minimize the loss function, this is all done in the backward pass of the training loop
and this is done by calling the backward() function on the loss, which computes the gradient of the loss function with respect to the parameters of the model using the chain rule of calculus, and then we are updating the parameters of the model using the optimizer.step() function

What is stochastic gradient descent? <br>
Stochastic Gradient Descent is a variant of the Gradient Descent algorithm, where instead of computing the gradient of the loss function with respect to the parameters using the entire training data, the gradient is computed using a single sample or a mini-batch of samples. This makes the computation faster and more efficient, especially for large datasets

Some best resources to understand better:<br>
* https://www.youtube.com/watch?v=IHZwWFHWa-w&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=2
* https://www.youtube.com/watch?v=RVwcBGzQcT8
* https://www.youtube.com/watch?v=b1lINT4u3qs
* https://www.youtube.com/watch?v=Ilg3gGewQ5U&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=4


Why do we need train loss and test loss?<br>
We need train loss to check how well the model is learning the training data, and we need test loss to check how well the model is generalizing on the unseen data, if the test loss is high, then the model is overfitting

Why is model overfitting or underfitting?<br>
Model is overfitting when the model is learning the training data very well but it is not generalizing on the unseen data, and model is underfitting when the model is not learning the training data well, in this case, we need to increase the model complexity

Why do we need to use torch.no_grad() or torch.inference_mode()? <br>
torch.no_grad() is used to disable the gradient computation, so that the model does not update the weights during the testing phase
torch.inference_mode() is used to disable the gradient computation, and also disables the autograd engine, which provides significant performance improvement during inference
We use torch.no_grad() or torch.inference_mode() during testing or inference to speed up the computation and to save memory

What does model.eval() do?<br>
model.eval() sets the model to evaluation mode, it is used to set the model to evaluation mode because some layers like Dropout and BatchNorm behave differently during training and testing, so we need to set the model to evaluation mode before testing the model

In [ ]:
# Predictions on the test data
model.eval()
with torch.inference_mode():
    y_test_pred = model(x_test)
    plot_graph(x_train, y_train, x_test, y_test, y_test_pred.detach())

# Saving the modal for future use

When saving a model for inference, it is only necessary to save the trained model’s learned parameters. Saving the model’s state_dict with the torch.save() function will give you the most flexibility for restoring the model later, which is why it is the recommended method for saving models.

A common PyTorch convention is to save models using either a .pt or .pth file extension.

Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.


# Some extra and indepth analysis

#### Let's break down the statement: "Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference."

##### Model Evaluation:
In deep learning, training and inference are two distinct phases. During training, the model learns from the data by adjusting its parameters to minimize the loss function. Inference, on the other hand, is the phase where the trained model makes predictions on new, unseen data.
During inference, we want the model to make predictions accurately without updating its parameters. Therefore, we need to ensure that the model behaves consistently and doesn't apply techniques like dropout or batch normalization, which are specific to training.
Dropout:

##### Dropout
Dropout is a regularization technique used during training to prevent overfitting. It works by randomly setting a fraction of input units to zero during each forward pass. This prevents units from co-adapting too much and forces the network to learn more robust features.
During inference, we want to utilize the full capacity of the trained model without randomly dropping units. Therefore, we disable dropout during inference to ensure consistent predictions.
Batch Normalization:

##### Batch Normalization
Batch normalization is a technique used to improve the training speed, stability, and performance of deep neural networks. It normalizes the activations of each layer by adjusting and scaling the outputs to have zero mean and unit variance.
During training, batch normalization uses the mean and variance of each batch to normalize the activations. However, during inference, we typically don't have access to batches of data. Therefore, we use the running estimates of mean and variance computed during training to normalize activations during inference.
Setting to Evaluation Mode:

##### Setting to Evaluation Mode:
`model.eval()` is a method provided by PyTorch (or other deep learning frameworks) to set the model to evaluation mode. When called, it sets the model to a mode suitable for inference.
When a model is in evaluation mode, it disables certain operations that are specific to training, such as dropout and batch normalization. Instead, it ensures that the model behaves consistently and makes predictions without modifying its parameters.
In summary, calling `model.eval()` before running inference ensures that the model behaves consistently during inference by disabling training-specific techniques like dropout and batch normalization. This ensures that the model makes predictions accurately and reliably on new data.

<br>

And these dropouts and batch normalization are used to prevent overfitting behing the scenes by pytorch and we can also use them explicitly in our model to prevent overfitting



## Intro to Batch Normalization and it's effect on internal covariate shift

* https://www.analyticsvidhya.com/blog/2021/03/introduction-to-batch-normalization/

Question:
Why does having mean and variance of 0 and 1 help in training the model? How did researchers come up with this 0 and 1 normalization?

## Difference between gradient descent and stochastic gradient descent

Gradient Descent
In gradient descent, the gradient is calculated for the entire dataset and the weights are updated based on the average gradient of the entire dataset. This can be computationally expensive for large datasets.
    
Stochastic Gradient Descent
In stochastic gradient descent, the gradient is calculated for each data point and the weights are updated based on the gradient of each data point. This can be computationally efficient for large datasets as it updates the weights more frequently.
    
Mini-batch Gradient Descent
Mini-batch gradient descent is a compromise between gradient descent and stochastic gradient descent. In mini-batch gradient descent, the gradient is calculated for a small batch of data points and the weights are updated based on the average gradient of the batch. This can be computationally efficient and can provide a balance between the two extremes.
    
Mini-batch gradient descent is commonly used in practice as it provides a good balance between efficiency and accuracy.

# Saving the model

Important methods:<br>

`.save()` - Saves the model to the disk<br>
`.load_state_dict()` - Loads the model from the disk<br>
`.state_dict()` - Returns the state of the model


In [ ]:
# Save the model
torch.save(model.state_dict(), 'linear_modal.pth')

In [ ]:
# Load the model
model = LinearRegression()
model.load_state_dict(torch.load('linear_modal.pth'))

# Predictions on the test data
model.eval()
with torch.inference_mode():
    y_test_pred = model(x_test)
    plot_graph(x_train, y_train, x_test, y_test, y_test_pred.detach())

Some of the ways to improve the model
1. Increase the number of epochs
2. Increase the learning rate
3. Change the optimizer
4. Change the loss function
5. Change the model architecture
6. Add more data
7. Add more features
8. Add more layers to the neural network
9. Change the activation function

How do neural networks learn the weights and biases?<br>
Neural networks learn the weights and biases by minimizing the loss function using the optimization algorithms like Stochastic Gradient Descent (SGD), Adam, etc. The loss function is the measure of the difference between the actual output and the predicted output. The optimization algorithm updates the weights and biases in the direction that minimizes the loss function.

But if we only use linear layers, then the output will be linear, so what is the use of neural networks?<br>
Neural networks are used to learn complex patterns in the data. Even though the output of the linear layer is linear, by stacking multiple linear layers with non-linear activation functions like ReLU, Sigmoid, etc., we can learn complex patterns in the data. This is the reason why neural networks are used for complex tasks like image classification, object detection, etc.

How does activation function help in learning complex patterns?<br>
Activation functions introduce non-linearity to the model. Without activation functions, the output of the model will be a linear combination of the input features. By introducing non-linearity, the model can learn complex patterns in the data. For example, ReLU activation function introduces non-linearity by setting all negative values to zero. This helps the model to learn complex patterns in the data.

How does the activation function introduce non-linearity?<br>
Activation functions introduce non-linearity by applying a non-linear transformation to the input. For example, ReLU activation function applies the transformation f(x) = max(0, x) to the input. This transformation introduces non-linearity to the model by setting all negative values to zero. This helps the model to learn complex patterns in the data.

So is that like modal is learning by mixing the linear and non-linear functions? and how it is learning complex patterns?<br>
Yes, neural networks learn complex patterns by mixing linear and non-linear functions. By stacking multiple linear layers with non-linear activation functions like ReLU, Sigmoid, etc., the model can learn complex patterns in the data. The linear layers learn the linear patterns in the data, while the non-linear activation functions introduce non-linearity to the model, allowing it to learn complex patterns.

Can you explain it with an example?<br>
Sure, let's take an example of image classification. In image classification, the input is an image, which is a 2D array of pixel values. The linear layers learn the linear patterns in the pixel values, while the non-linear activation functions like ReLU, Sigmoid, etc., introduce non-linearity to the model, allowing it to learn complex patterns like edges, textures, shapes, etc. By stacking multiple linear layers with non-linear activation functions, the model can learn complex patterns in the image data, enabling it to classify the images into different classes.


More info : 
* https://www.youtube.com/watch?v=0QczhVg5HaI
* https://www.youtube.com/watch?v=TkwXa7Cvfr8


# Extra Reading

How to decide which one is the best GPU for deep learning?<br>
https://timdettmers.com/2023/01/30/which-gpu-for-deep-learning/

Sometimes training time on CPU is faster than GPU, why? <br>
Training time on CPU is faster than GPU when the model is very small, because the overhead of transferring the data to the GPU and back to the CPU is more than the time taken to train the model on the CPU.
Also, when the model is very small, the parallelism provided by the GPU is not utilized properly, which makes the training time on the CPU faster than the GPU.

For efficient usage of GPU see: https://horace.io/brrr_intro.html